## 7.1 투표 기반 분류기

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 직접 투표 (hard voting)
- 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측
- 다수결 투표로 정해지는 분류기

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

"""
여러 분류기를 조합하여 투표 기반 분류기(VotingClassifier)를 만든 후, 훈련
"""

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
"""
각 분류기의 테스트셋 정확도 확인
=> 투표 기반 분류기(voting)가 다른 개별 분류기(log, rnd, svm)보다 성능이 조금 더 높음.
"""

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.896


## 간접 투표 (soft voting)
- 모든 분류기가 클래스의 확률을 예측할 수 있으면(즉, predict_proba() 메서드가 있으면), 개별 투표기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측
- 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다.
    - but, 확률을 추정하기 위해 교차 검증을 사용하므로 훈련 속도가 느려진다.

In [4]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [5]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.92


## 7.2 배깅과 페이스팅
- 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시킨다.
- 배깅(bagging) : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
    - 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있다.
- 페이스팅(pasting) : 중복을 허용하지 않고 샘플링하는 방식

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

"""
결정 트리 분류기 500개의 앙상블을 훈련시키는 코드.
각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련된다.
=> 이는 배깅의 경우이고, 페이스팅을 사용하려면 bootstrap=False로 지정하면 된다.
"""
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## 7.5 부스팅
- 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법.
- 앞의 모델을 보완해나가면서 일련의 예측기를 학습.
- 가장 인기 있는 모델 -> AdaBoost, Gradient Boosting

### 에이다부스트
- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것.
    - 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 된다.
- 단점 : 연속된 학습 기법을 하면 각 예측기는 이전 예측기가 훈련되고 평가된 후에 학습할 수 있기 때문에 병렬화(또는 분할)를 할 수 없다. 결국 배깅이나 페이스팅만큼 확장성이 높지 않다.

In [7]:
"""
200개의 아주 얕은 결정 트리를 기반으로 하는 에이다부스트 분류기를 훈련
-> 결정 트리는 max_depth=1 (즉, 결정 노드 하나와 리프 노드 두 개로 이루어진 트리)

에이다부스트 앙상블이 훈련 세트에 과대적합되면 추정기 수를 줄이거나, 추정기의 규제를 더 강하게 할 것.
"""

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 그레이디언트 부스팅
- 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 residual error(잔여 오차)에 새로운 예측기를 학습시킨다.


In [10]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

np.random.seed(42)
# 간단한 이차식 형태의 데이터셋
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

# DecisionTreeRegressor를 훈련 세트에 학습
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

# 첫 번쨰 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor를 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

# 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

# 세 개의 트리를 포함하는 앙상블 모델 생성
# 새로운 샘플에 대한 예측을 만들기 위해, 모든 트리의 예측을 더한다.
X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))